In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
uchicago = pd.read_csv("season_plays23.csv").iloc[:, 1:]
w_uchicago = pd.read_csv("w_season_plays23.csv").iloc[:, 1:]

In [3]:
def points_scored(action):
    if "3-pt" in action:
        return 3
    elif "free throw" in action:
        return 1
    else:
        return 2
    
def calculate_scores(df, lineup):
    current = df[df.Lineup == lineup]
    
    makes = current[current.Action.str.contains('made')]
    makes["Points"] = makes.Action.apply(points_scored)

    chicago_scores = makes[makes.Chicago == makes.Team].Points.sum()
    other_scores = makes[makes.Chicago != makes.Team].Points.sum()

    return (chicago_scores, other_scores)

def calculate_play(df, lineup, play):
    current = df[df.Lineup == lineup]
    plays = current[(current.Action.str.contains(play))]

    chicago = plays[(plays.Chicago == plays.Team)].shape[0]
    other = plays[(plays.Chicago != plays.Team)].shape[0]
    
    return (chicago, other)

def calculate_makes(df, lineup, three=False, ft=False):
    current = df[df.Lineup == lineup]
    
    makes = current[current.Action.str.contains("made")]
    if three:
        makes = makes[makes.Action.str.contains("3-pt")]
    elif ft:
        makes = makes[makes.Action.str.contains("free throw")]
    else:
        makes = makes[~makes.Action.str.contains("free throw")]
    
    chicago_makes = makes[(makes.Chicago == makes.Team)].shape[0]
    other_makes = makes[(makes.Chicago != makes.Team)].shape[0]
    
    return (chicago_makes, other_makes)

def calculate_percentage(df, lineup, three=False, ft=False, att=False):
    current = df[df.Lineup == lineup]
    misses = current[(current.Action.str.contains("missed"))]
    makes = current[(current.Action.str.contains("made"))]
    
    if three:
        misses = misses[misses.Action.str.contains("3-pt")]
        makes = makes[makes.Action.str.contains("3-pt")]
    elif ft:
        misses = misses[misses.Action.str.contains("free throw")]
        makes = makes[makes.Action.str.contains("free throw")]
    else:
        misses = misses[~(misses.Action.str.contains("free throw"))]
        makes = makes[~(makes.Action.str.contains("free throw"))]
        
    chicago_misses = misses[(misses.Chicago == misses.Team)].shape[0]
    chicago_makes = makes[(makes.Chicago == makes.Team)].shape[0]
    other_misses = misses[(misses.Chicago != misses.Team)].shape[0]
    other_makes = makes[(makes.Chicago != makes.Team)].shape[0]
    
    chicago_attempts = chicago_misses + chicago_makes
    other_attempts = other_misses + other_makes
    
    if att:
        return (chicago_attempts, other_attempts)
    
    if (chicago_attempts == 0) and (other_attempts == 0):
        return (0, 0)
    elif chicago_attempts == 0:
        return (0, other_makes / other_attempts)
    elif other_attempts == 0:
        return (chicago_makes / chicago_attempts, 0)
    else:
        return (chicago_makes / chicago_attempts, 
                other_makes / other_attempts)
    
def sec_to_time(seconds):
    minutes = str(seconds // 60)
    
    if (seconds % 60) < 10:
        return minutes + ":0" + str(seconds % 60)
    else:
        return minutes + ":" + str(seconds % 60)

def time_passed(start, end):
    if start == '30300':
        start = "00:30"
    elif end == '30300':
        end = "00:30"
    
    start_time = int(start.split(":")[0])*60 + int(start.split(":")[1])
    end_time = int(end.split(":")[0])*60 + int(end.split(":")[1])
    
    if start_time > end_time:
        return start_time - end_time
    else:
        return 0

def calculate_time_played(df, lineup):
    total_time = 0
    for x in range(len(df[df.Lineup == lineup])-1):
        current_play = df[df.Lineup == lineup].reset_index(drop=False).values[x]
        next_play = df[df.Lineup == lineup].reset_index(drop=False).values[x+1]
        if x > 0:
            last_play = df[df.Lineup == lineup].reset_index(drop=False).values[x-1]
        else:
            last_play = current_play

        if (current_play[0] != next_play[0] - 1) or (current_play[0] == 342):
            continue
            
        if (current_play[0] == 0) or (last_play[4] != current_play[4]):
            continue
        elif next_play[4] != current_play[4]:
            start = current_play[5]
            end = "00:00"
        else:
            start = current_play[5]
            end = next_play[5]

        total_time += time_passed(start, end)

    return total_time / 60

def analyze_lineup(df, lineup):
    return [calculate_time_played(df, lineup), # MP
            
            calculate_scores(df, lineup)[0], # PTS
            calculate_scores(df, lineup)[1], # OPP PTS
            
            calculate_scores(df, lineup)[0] - calculate_scores(df, lineup)[1], # +/-
            
            calculate_play(df, lineup, "Assist")[0], # AST
            
            calculate_play(df, lineup, "offensive rebound")[0], # OREB
            calculate_play(df, lineup, "offensive rebound")[1], # OPP OREB
            calculate_play(df, lineup, "defensive rebound")[0], # DREB
            calculate_play(df, lineup, "defensive rebound")[1], # OPP DREB
            
            calculate_makes(df, lineup)[0], # FGM
            calculate_percentage(df, lineup, att=True)[0], # FGA
            calculate_makes(df, lineup)[1], # OPP FGM
            calculate_percentage(df, lineup, att=True)[1], # OPP FGA
            
            calculate_makes(df, lineup, three=True)[0], # 3PM
            calculate_percentage(df, lineup, three=True, att=True)[0], # 3PA
            calculate_makes(df, lineup, three=True)[1], # OPP 3PM
            calculate_percentage(df, lineup, three=True, att=True)[1], # OPP 3PA
            
            calculate_makes(df, lineup, ft=True)[0], # FTM
            calculate_makes(df, lineup, ft=True)[1], # OPP FTM
            calculate_percentage(df, lineup, ft=True, att=True)[0], # FTA
            calculate_percentage(df, lineup, ft=True, att=True)[1], # OPP FTA
            
            calculate_play(df, lineup, "Turnover")[0], # TO
            calculate_play(df, lineup, "Turnover")[1],# OPP TO
            
            calculate_play(df, lineup, "Steal")[0], # STL
            calculate_play(df, lineup, "Block")[0], # BLK
            
            calculate_play(df, lineup, "Foul")[0], # PF
            calculate_play(df, lineup, "Foul")[1]] # OPP PF

def pbp_lineups(game_pbp):
    date = game_pbp.Date.values[0]
    lineups = [l for l in pd.unique(game_pbp.Lineup) if len(l.split(", ")) == 5]

    lineup_data = []
    for line in lineups:
        lineup_data.append( [line] + analyze_lineup(game_pbp, line) )

    lineup_analysis = pd.DataFrame(lineup_data, 
                                   columns = ['Lineup',
                                              'MP', 
                                              'PTS', 'OPP PTS', 
                                              '+/-', 
                                              'AST',
                                              'OREB', 'OPP OREB', 'DREB', 'OPP DREB',
                                              'FGM', 'FGA', 'OPP FGM', 'OPP FGA',
                                              '3PM', '3PA', 'OPP 3PM', 'OPP 3PA',
                                              'FTM', 'OPP FTM', 'FTA', 'OPP FTA', 
                                              'TO', 'OPP TO', 
                                              'STL', 'BLK',
                                              'PF', 'OPP PF'])
    
    lineup_analysis['Lineup_Set'] = lineup_analysis.Lineup.apply(lambda i : set(i.split(", ")))
    lineup_analysis['Lineup'] = lineup_analysis['Lineup_Set'].apply(lambda i : ", ".join(sorted(i)))
    lineup_analysis = lineup_analysis.groupby('Lineup').sum().reset_index(drop=False)
    
    return lineup_analysis

In [4]:
def add_advanced(lineup_analysis):
    lineup_analysis['FG%'] = 100 * lineup_analysis['FGM'] / lineup_analysis['FGA']
    lineup_analysis['OPP FG%'] = 100 * lineup_analysis['FGM'] / lineup_analysis['FGA']
    lineup_analysis['3P%'] = 100 * lineup_analysis['3PM'] / lineup_analysis['3PA']
    lineup_analysis['OPP 3P%'] = 100 * lineup_analysis['OPP 3PM'] / lineup_analysis['OPP 3PA']
    lineup_analysis['FT%'] = 100 * lineup_analysis['FTM'] / lineup_analysis['FTA']
    lineup_analysis['OPP FT%'] = 100 * lineup_analysis['OPP FTM'] / lineup_analysis['OPP FTA']
    
    # formula from nbastuffer.com
    lineup_analysis['POSS'] = 0.96 * (lineup_analysis.FGA +
                                      lineup_analysis.TO + 
                                      0.44*(lineup_analysis.FTA) - 
                                      lineup_analysis.OREB)
    lineup_analysis['OPP POSS'] = 0.96 * (lineup_analysis['OPP FGA'] +
                                          lineup_analysis['OPP TO'] + 
                                          0.44*(lineup_analysis['OPP FTA']) - 
                                          lineup_analysis['OPP OREB'])

    lineup_analysis['PPP'] = lineup_analysis.PTS / lineup_analysis.POSS
    lineup_analysis['OPP PPP'] = lineup_analysis['OPP PTS'] / lineup_analysis['OPP POSS']
    
    lineup_analysis['FTA%'] = 100 * lineup_analysis['FTA'].astype(int) / lineup_analysis['FGA'].astype(int)
    lineup_analysis['OPP FTA%'] = 100 * lineup_analysis['OPP FTA'].astype(int) / lineup_analysis['OPP FGA'].astype(int)

    lineup_analysis['PTS DIFF PER40'] = 40 * (lineup_analysis['PTS'] - lineup_analysis['OPP PTS']) / lineup_analysis['MP']

    lineup_analysis['REB'] = lineup_analysis.OREB + lineup_analysis.DREB
    lineup_analysis['OPP REB'] = lineup_analysis['OPP OREB'] + lineup_analysis['OPP DREB']
    lineup_analysis['ORB%'] = 100 * lineup_analysis.OREB / (lineup_analysis.OREB + lineup_analysis['OPP DREB'])
    lineup_analysis['DRB%'] = 100 * lineup_analysis.DREB / (lineup_analysis.DREB + lineup_analysis['OPP OREB'])
    lineup_analysis['RB%'] = 100 * lineup_analysis.REB / (lineup_analysis.REB + lineup_analysis['OPP REB'])
    lineup_analysis['OPP ORB%'] = 100 * lineup_analysis['OPP OREB'] / (lineup_analysis['OPP OREB'] + lineup_analysis.DREB)
    lineup_analysis['OPP DRB%'] = 100 * lineup_analysis['OPP DREB'] / (lineup_analysis['OPP DREB'] + lineup_analysis.OREB)
    lineup_analysis['OPP RB%'] = 100 * lineup_analysis['OPP REB'] / (lineup_analysis['OPP REB'] + lineup_analysis.REB)

    lineup_analysis['TO%'] = 100 * lineup_analysis.TO / lineup_analysis.POSS
    lineup_analysis['OPP TO%'] = 100 * lineup_analysis['OPP TO'] / lineup_analysis['OPP POSS']
    
    lineup_analysis['POSS PER40'] = 40 * lineup_analysis['POSS'] / lineup_analysis['MP']
    
    lineup_analysis = lineup_analysis.round(2)
    lineup_analysis['POSS'] = lineup_analysis['POSS'].round(1)
    lineup_analysis['OPP POSS'] = lineup_analysis['OPP POSS'].round(1)
    lineup_analysis['ORB%'] = lineup_analysis['ORB%'].round(1)
    lineup_analysis['DRB%'] = lineup_analysis['DRB%'].round(1)
    lineup_analysis['RB%'] = lineup_analysis['RB%'].round(1)
    lineup_analysis['TO%'] = lineup_analysis['TO%'].round(1)
    lineup_analysis['OPP ORB%'] = lineup_analysis['OPP ORB%'].round(1)
    lineup_analysis['OPP DRB%'] = lineup_analysis['OPP DRB%'].round(1)
    lineup_analysis['OPP RB%'] = lineup_analysis['OPP RB%'].round(1)
    lineup_analysis['OPP TO%'] = lineup_analysis['OPP TO%'].round(1)
    lineup_analysis['FTA%'] = lineup_analysis['FTA%'].round(1)
    lineup_analysis['OPP FTA%'] = lineup_analysis['OPP FTA%'].round(1)
    
    return lineup_analysis

def pbp_helper(df):
    lineup_data = []
    for x in pd.unique(df.Opponent):
        lineup_data.append( pbp_lineups( df[(df.Opponent == x)] ) )
        
    lineup_analysis = pd.concat(lineup_data)
    lineup_analysis = lineup_analysis.groupby('Lineup').sum()
        
    lineup_analysis = add_advanced(lineup_analysis)
    
    lineup_analysis = lineup_analysis.fillna(0).sort_values(by='MP', ascending=False)
    
    lineup_analysis['Lineup'] = lineup_analysis.index
    lineup_analysis.Lineup = lineup_analysis.Lineup.apply(lambda l : ", ".join([i.split(" ")[0] for i in l.split(", ")]))
    
    return lineup_analysis.set_index('Lineup')

def win_loss(margin):
    if margin > 0:
        return "W"
    else:
        return "L"

def game_helper( game_pbp ):
    df = game_pbp.copy()
    df['Lineup'] = df.Game.apply(lambda i : ", ".join([str(i)] * 5))

    lineup_data = []
    for x in pd.unique(df.Game):
        lineup_data.append( [pbp_lineups( df[(df.Game == x)] ), x] )

    lineup_analysis = pd.concat( [i[0] for i in lineup_data] )
    lineup_analysis['Game'] = [i[1] for i in lineup_data]

    lineup_analysis = add_advanced(lineup_analysis)
    lineup_analysis['Lineup'] = list(pd.DataFrame(game_pbp.groupby(by=['Date', 'Opponent'])['Game'].first()).sort_values(by=['Game']).reset_index(drop=False).Opponent)

    lineup_analysis = lineup_analysis.set_index("Lineup")

    lineup_analysis['Margin'] = (lineup_analysis['PTS'].astype(int) - lineup_analysis['OPP PTS'].astype(int))
    lineup_analysis['W/L'] = lineup_analysis.Margin.apply(win_loss)

    lineup_analysis = lineup_analysis[['W/L', 'Margin', 'POSS', 'OPP POSS', 'PPP', 'OPP PPP', 'FTA%', 'OPP FTA%', 'PTS DIFF PER40', 'REB', 'OPP REB', 'ORB%', 'DRB%', 'RB%', 'TO%', 'OPP TO%']]

    return lineup_analysis.fillna(0)

def total_advanced(game_pbp, uaa=False):
    df = game_pbp.copy()
    
    if uaa:
        df = df[(df.Opponent.isin(['Washington-St. Louis', 'New York U.', 'Brandeis', 
                                   'Carnegie Mellon', 'Case Western Reserve',
                                   'Emory', 'Rochester (N.Y.)']))]

    df['Lineup'] = ", ".join(['Total']*5)
    
    lineup_analysis = pbp_lineups(df)
    
    lineup_analysis = add_advanced(lineup_analysis)
    
    lineup_analysis['POSS'] = round(40 * lineup_analysis['POSS'] / lineup_analysis['MP'], 1)
    lineup_analysis['OPP POSS'] = round(40 * lineup_analysis['OPP POSS'] / lineup_analysis['MP'], 1)
    lineup_analysis['W/L'] = ""
    lineup_analysis['Margin'] = lineup_analysis['PTS'].astype(int) - lineup_analysis['OPP PTS'].astype(int)
    
    lineup_analysis = lineup_analysis[['W/L', 'Margin', 'POSS', 'OPP POSS', 'PPP', 'OPP PPP', 'FTA%', 'OPP FTA%', 'PTS DIFF PER40', 'REB', 'OPP REB', 'ORB%', 'DRB%', 'RB%', 'TO%', 'OPP TO%']]
    lineup_analysis.index = ['Total']
    
    return lineup_analysis

In [5]:
team_start = "10:00"
current = w_uchicago
date = "March 4, 2023"

new = current[(current.Date == date)]
new['length'] = new.Lineup.apply(lambda i : len(i.split(", ")))

print(pbp_helper( current[(current.Date == date)] ).MP.sum())
display(new[(new.length != 5) & (new.Substitution == 0)])

39.17


,Date,Chicago,Opponent,Half,Time,Score,Team,Action,Substitution,Player,Lineup,Points,Game,length


In [6]:
lineups = pbp_helper( current )

lineups

,MP,PTS,OPP PTS,+/-,AST,OREB,OPP OREB,DREB,OPP DREB,FGM,FGA,OPP FGM,OPP FGA,3PM,3PA,OPP 3PM,OPP 3PA,FTM,OPP FTM,FTA,OPP FTA,TO,OPP TO,STL,BLK,PF,OPP PF,FG%,OPP FG%,3P%,OPP 3P%,FT%,OPP FT%,POSS,OPP POSS,PPP,OPP PPP,FTA%,OPP FTA%,PTS DIFF PER40,REB,OPP REB,ORB%,DRB%,RB%,OPP ORB%,OPP DRB%,OPP RB%,TO%,OPP TO%,POSS PER40
Lineup,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Ellie, Grace, Mallory, Marissa, Sophia",286.30,538,429,109,94,96,81,207,189,188,463,160,442,47,146,41,150,115,68,151,98,123,135,61,21,107,144,40.60,40.60,32.19,27.33,76.16,69.39,534.2,517.6,1.01,0.83,32.6,22.2,15.23,303,270,33.7,71.9,52.9,28.1,66.3,47.1,23.0,26.1,74.63
"Bella, Ellie, Grace, Mallory, Marissa",56.60,117,73,44,22,11,7,46,35,37,80,28,80,13,30,6,32,29,11,36,13,22,27,17,4,21,27,46.25,46.25,43.33,18.75,80.56,84.62,102.6,101.5,1.14,0.72,45.0,16.2,31.10,57,42,23.9,86.8,57.6,13.2,76.1,42.4,21.4,26.6,72.49
"Ashley, Bella, Grace, Lindsey, Sophia",43.97,61,73,-12,7,11,10,37,38,24,72,26,70,2,17,8,20,11,13,17,19,14,17,8,4,20,18,33.33,33.33,11.76,40.00,64.71,68.42,79.2,82.0,0.77,0.89,23.6,27.1,-10.92,48,48,22.4,78.7,50.0,21.3,77.6,50.0,17.7,20.8,72.04
"Bella, Grace, Mallory, Marissa, Sophia",43.35,81,78,3,14,18,8,29,25,26,68,23,59,8,27,8,24,21,24,23,29,14,19,9,2,18,21,38.24,38.24,29.63,33.33,91.30,82.76,71.2,79.4,1.14,0.98,33.8,49.2,2.77,47,33,41.9,78.4,58.8,21.6,58.1,41.2,19.7,23.9,65.66
"Ellie, Grace, Lindsey, Marissa, Sophia",41.53,64,54,10,13,17,16,37,32,24,68,21,71,5,18,3,15,11,9,24,18,18,18,10,3,16,22,35.29,35.29,27.78,20.00,45.83,50.00,76.4,77.7,0.84,0.70,35.3,25.4,9.63,54,48,34.7,69.8,52.9,30.2,65.3,47.1,23.6,23.2,73.56
"Ashley, Bella, Grace, Mallory, Sophia",29.72,50,48,2,9,4,8,21,26,20,48,20,52,5,14,3,16,5,5,10,6,5,9,6,3,9,9,41.67,41.67,35.71,18.75,50.00,83.33,51.3,53.4,0.98,0.90,20.8,11.5,2.69,25,34,13.3,72.4,42.4,27.6,86.7,57.6,9.8,16.8,69.00
"Ashley, Ellie, Grace, Mallory, Sophia",29.20,51,43,8,14,3,8,19,15,22,39,12,40,3,11,5,14,4,14,7,18,8,13,5,5,19,12,56.41,56.41,27.27,35.71,57.14,77.78,45.2,50.8,1.13,0.85,18.0,45.0,10.96,22,23,16.7,70.4,48.9,29.6,83.3,51.1,17.7,25.6,61.91
"Ashley, Bella, Lindsey, Marissa, Sophia",25.52,61,25,36,14,11,6,20,14,22,42,10,34,5,10,1,12,12,4,17,7,5,15,7,2,9,10,52.38,52.38,50.00,8.33,70.59,57.14,41.7,44.2,1.46,0.57,40.5,20.6,56.43,31,20,44.0,76.9,60.8,23.1,56.0,39.2,12.0,33.9,65.43
"Ashley, Bella, Grace, Lindsey, Marissa",24.77,33,29,4,6,8,8,20,15,14,38,11,38,2,10,2,13,3,5,4,10,7,8,4,4,14,6,36.84,36.84,20.00,15.38,75.00,50.00,37.2,40.7,0.89,0.71,10.5,26.3,6.46,28,23,34.8,71.4,54.9,28.6,65.2,45.1,18.8,19.6,60.10


In [7]:
game_advanced = pd.concat([game_helper( current ), 
                           total_advanced( current, uaa=True ), 
                           total_advanced( current )])
game_advanced.index = list(game_advanced.index)[0:len(game_advanced.index)-2] + ["UAA Total", "Total"]

game_advanced

,W/L,Margin,POSS,OPP POSS,PPP,OPP PPP,FTA%,OPP FTA%,PTS DIFF PER40,REB,OPP REB,ORB%,DRB%,RB%,TO%,OPP TO%
North Central (Ill.),W,20,66.7,65.8,1.02,0.73,13.3,12.5,20.38,40,36,35.5,64.4,52.6,25.5,25.8
Benedictine (Ill.),W,28,60.7,60.5,1.05,0.60,56.6,44.6,28.16,53,33,48.6,70.6,61.6,23.1,18.2
North Park,W,23,71.8,71.6,1.00,0.68,30.3,24.6,23.26,49,38,29.3,80.4,56.3,16.7,22.3
Colorado College,W,11,77.2,79.7,1.07,0.90,46.4,36.8,11.09,36,36,17.2,72.1,50.0,23.3,20.1
Carroll (Wis.),W,9,71.1,72.6,0.98,0.84,31.4,22.4,9.08,39,34,22.2,71.7,53.4,31.0,20.7
Carthage,W,8,61.1,60.6,1.31,1.19,41.5,46.9,8.08,30,26,24.1,85.2,53.6,13.1,13.2
Eureka,W,23,65.7,68.4,1.08,0.70,47.3,38.2,23.41,38,37,36.4,61.9,50.7,21.3,33.6
Wheaton (Ill.),W,19,63.2,62.2,0.89,0.60,17.5,7.4,19.23,48,36,38.6,77.5,57.1,23.7,29.0
UW-Whitewater,W,5,65.1,66.2,0.97,0.88,39.2,16.1,5.04,33,36,28.1,64.9,47.8,26.1,33.2
Wis. Lutheran,W,5,63.6,63.8,0.93,0.85,23.1,30.9,5.04,41,30,28.1,82.0,57.8,28.3,17.2


In [8]:
games = []
for date in pd.unique(current.Date):
    opp_game = pbp_helper( current[(current.Date == date)] )
    opp_game['Opponent'] = current[(current.Date == date)].Opponent.values[0]
    games.append(opp_game)
    
games = pd.concat(games)

games['Lineup'] = games.index
games.Lineup = games.Lineup.apply(lambda l : ", ".join([i.split(" ")[0] for i in l.split(", ")]))

games = games.set_index('Lineup')

games

,MP,PTS,OPP PTS,+/-,AST,OREB,OPP OREB,DREB,OPP DREB,FGM,FGA,OPP FGM,OPP FGA,3PM,3PA,OPP 3PM,OPP 3PA,FTM,OPP FTM,FTA,OPP FTA,TO,OPP TO,STL,BLK,PF,OPP PF,FG%,OPP FG%,3P%,OPP 3P%,FT%,OPP FT%,POSS,OPP POSS,PPP,OPP PPP,FTA%,OPP FTA%,PTS DIFF PER40,REB,OPP REB,ORB%,DRB%,RB%,OPP ORB%,OPP DRB%,OPP RB%,TO%,OPP TO%,POSS PER40,Opponent
Lineup,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Ellie, Grace, Mallory, Marissa, Sophia",7.98,14,10,4,2,2,5,4,3,5,10,4,13,1,1,0,1,3,2,4,2,4,4,3,2,2,2,50.00,50.00,100.00,0.00,75.00,100.00,13.2,12.4,1.06,0.81,40.0,15.4,20.04,6,8,40.0,44.4,42.9,55.6,60.0,57.1,30.3,32.4,66.19,North Central (Ill.)
"Bella, Ellie, Grace, Mallory, Marissa",6.95,12,14,-2,3,1,2,3,4,5,10,6,11,2,4,2,3,0,0,0,0,2,3,1,1,1,3,50.00,50.00,50.00,66.67,0.00,0.00,10.6,11.5,1.14,1.22,0.0,0.0,-11.51,4,6,20.0,60.0,40.0,40.0,80.0,60.0,18.9,26.0,60.78,North Central (Ill.)
"Ashley, Bella, Grace, Lindsey, Sophia",6.05,3,11,-8,0,2,3,5,5,1,8,4,13,0,2,3,5,1,0,2,0,2,2,1,1,3,2,12.50,12.50,0.00,60.00,50.00,0.00,8.5,11.5,0.35,0.95,25.0,0.0,-52.89,7,8,28.6,62.5,46.7,37.5,71.4,53.3,23.5,17.4,56.36,North Central (Ill.)
"Bella, Grace, Lindsey, Marissa, Sophia",4.03,8,5,3,1,1,1,5,1,4,6,2,8,0,1,1,3,0,0,0,0,4,1,0,1,1,1,66.67,66.67,0.00,33.33,0.00,0.00,8.6,7.7,0.93,0.65,0.0,0.0,29.75,6,2,50.0,83.3,75.0,16.7,50.0,25.0,46.3,13.0,85.69,North Central (Ill.)
"Ashley, Bella, Isabelle, Lindsey, Marissa",2.43,0,1,-1,0,2,1,2,1,0,4,0,4,0,3,0,2,0,1,0,2,1,1,0,0,2,0,0.00,0.00,0.00,0.00,0.00,50.00,2.9,4.7,0.00,0.21,0.0,50.0,-16.44,4,2,66.7,66.7,66.7,33.3,33.3,33.3,34.7,21.4,47.34,North Central (Ill.)
"Ashley, Bella, Lindsey, Marissa, Sophia",2.30,10,0,10,3,0,2,2,1,5,5,0,4,0,0,0,2,0,0,1,0,0,2,2,0,0,1,100.00,100.00,0.00,0.00,0.00,0.00,5.2,3.8,1.91,0.00,20.0,0.0,173.91,2,3,0.0,50.0,40.0,50.0,100.0,60.0,0.0,52.1,90.82,North Central (Ill.)
"Ashley, Bella, Grace, Lindsey, Marissa",1.50,4,0,4,0,2,2,2,1,2,5,0,4,0,2,0,1,0,0,0,0,0,0,0,1,2,0,40.00,40.00,0.00,0.00,0.00,0.00,2.9,1.9,1.39,0.00,0.0,0.0,106.67,4,3,66.7,50.0,57.1,50.0,33.3,42.9,0.0,0.0,76.80,North Central (Ill.)
"Ellie, Grace, Lindsey, Marissa, Sophia",1.20,3,0,3,1,0,0,1,1,1,2,0,1,1,1,0,1,0,0,0,0,1,1,0,0,0,0,50.00,50.00,100.00,0.00,0.00,0.00,2.9,1.9,1.04,0.00,0.0,0.0,100.00,1,1,0.0,100.0,50.0,0.0,100.0,50.0,34.7,52.1,96.00,North Central (Ill.)
"Ashley, Bella, Grace, Mallory, Marissa",0.80,3,0,3,0,0,0,1,0,1,1,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,1,100.00,100.00,0.00,0.00,100.00,0.00,2.3,1.0,1.28,0.00,100.0,0.0,150.00,1,0,0.0,100.0,100.0,0.0,0.0,0.0,42.7,0.0,117.12,North Central (Ill.)
